# 🧠 Módulo 5: Memória e Persistência no Agno 2.0

## Construindo Agentes com Memória de Longo Prazo

---

**Bem-vindos ao Módulo 5!** 🎉

Até agora, nossos agentes eram meio esquecidos, né? Era como conversar com alguém que tem amnésia - toda vez que você volta a falar, ele não lembra de nada do que rolou antes!

**Hoje vamos resolver isso!** Vamos aprender como dar memória de longo prazo para nossos agentes, fazer eles lembrarem das conversas anteriores e até mesmo personalizar as respostas baseado no que já sabem sobre cada usuário.

**O que vamos aprender hoje:**
- 🧠 Sistema de memória do Agno
- 💾 Armazenamento de sessões
- 🔄 Continuidade de conversas
- 👤 Memórias por usuário
- 📊 Diferentes tipos de bancos de dados

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-05_img_01.png)

## 🎯 Por que Memória é Importante?

Imagina que você tem um assistente pessoal incrível, mas toda vez que você fala com ele, ele esqueceu tudo que vocês conversaram antes. Seria bem frustrante, né?

**É exatamente isso que acontece com agentes sem memória!**

### Problemas dos Agentes Esquecidos:
- 😵 Não lembram do contexto anterior
- 🔄 Usuário precisa repetir informações
- 🚫 Não conseguem personalizar respostas
- 📉 Experiência do usuário ruim

### Benefícios da Memória:
- 🧠 **Contexto contínuo**: O agente lembra da conversa
- 👤 **Personalização**: Adapta-se ao usuário
- 🎯 **Experiência melhor**: Conversas mais naturais
- 📈 **Aprendizado**: Melhora com o tempo

## 🚀 Setup Inicial

Vamos começar instalando as dependências e configurando nosso ambiente!

In [ ]:
# Instalando as dependências
!pip install agno google-generativeai sqlalchemy -q

print("📦 Dependências instaladas com sucesso!")
print("🎯 Agora vamos configurar nossa API key do Google AI Studio")

In [ ]:
import os
from getpass import getpass

# Configurando a API key do Google AI Studio
# Você pode pegar sua chave gratuita em: https://aistudio.google.com/app/apikey

api_key = getpass("🔑 Cole sua API key do Google AI Studio: ")
os.environ["GOOGLE_API_KEY"] = api_key

print("✅ API key configurada!")
print("🔥 Bora começar a criar agentes com memória!")

## 🧠 Conceitos Fundamentais: Sessões e Memória

Antes de meter a mão no código, vamos entender os conceitos principais:

### 📝 **Sessão (Session)**
É como uma "conversa completa" entre você e o agente. Tipo quando você abre o WhatsApp com alguém - tudo que vocês conversaram fica ali, né?

### 🔄 **Run**
Cada mensagem individual dentro de uma sessão. É como cada balãozinho de conversa no WhatsApp.

### 💾 **Storage**
O "cérebro" onde tudo fica guardado. Pode ser um banco de dados SQLite, PostgreSQL, ou até mesmo um arquivo JSON.

### 👤 **Memória do Usuário**
Informações específicas sobre cada pessoa que conversa com o agente. Tipo "João gosta de pizza" ou "Maria mora em São Paulo".

```mermaid
graph TD
    A[👤 Usuário] -->|Mensagem| B[🤖 Agente]
    B -->|Processa| C[🧠 Memória]
    C -->|Busca contexto| D[💾 Storage]
    D -->|Retorna histórico| C
    C -->|Resposta personalizada| B
    B -->|Resposta| A
    B -->|Salva interação| D
```

## 💾 Storage: O Cérebro do Agente

O **Storage** é onde toda a magia acontece! É como se fosse o "HD" do nosso agente.

No Agno 2.0, temos várias opções de storage:

### 🗃️ Tipos de Storage:
- **SQLite**: Perfeito para desenvolvimento e projetos pequenos
- **PostgreSQL**: Para produção e projetos grandes
- **MySQL**: Alternativa popular ao PostgreSQL
- **MongoDB**: Para dados não-relacionais
- **Redis**: Para cache e dados temporários
- **JSON**: Para testes e protótipos

**Dica!** 💡 Para começar, vamos usar SQLite porque é super simples - não precisa instalar nada!

In [ ]:
# Vamos importar tudo que precisamos
from agno.agent import Agent
from agno.models.google import Gemini
from agno.db.sqlite import SqliteDb
import os

# Criando nosso primeiro storage com SQLite
# É como criar um "cérebro" para o agente guardar as memórias
db = SqliteDb(db_file="meu_agente_memoria.db")

print("💾 Storage SQLite criado!")
print("📂 Arquivo: meu_agente_memoria.db")
print("🧠 Agora nosso agente pode ter memória!")

## 🤖 Primeiro Agente com Memória

Bora criar nosso primeiro agente que lembra das conversas! É como dar um upgrade de memória no nosso robô.

In [ ]:
# Criando um agente com memória
agente_com_memoria = Agent(
    name="Assistente Memória",
    model=Gemini(id="gemini-2.0-flash-exp"),
    
    # 🧠 Aqui está a magia! Conectamos o storage
    db=db,
    
    # 🔄 Isso faz o agente lembrar das conversas anteriores
    add_history_to_context=True,
    
    # 📊 Quantas interações anteriores ele deve lembrar
    num_history_runs=5,
    
    description="Sou um assistente que lembra de tudo que conversamos!",
    markdown=True
)

print("🎉 Agente com memória criado!")
print("🧠 Ele vai lembrar de tudo que vocês conversarem")
print("🔥 Vamos testar!")

In [ ]:
# Primeira conversa - vamos nos apresentar
print("💬 PRIMEIRA INTERAÇÃO:")
print("=" * 50)

response = agente_com_memoria.run(
    "Oi! Meu nome é João e eu sou desenvolvedor Python. Gosto muito de pizza e moro em São Paulo.",
    session_id="conversa_joao",  # ID da sessão para identificar a conversa
    user_id="joao123"  # ID do usuário
)

print(response.content)

In [ ]:
# Segunda interação - vamos ver se ele lembrou!
print("💬 SEGUNDA INTERAÇÃO:")
print("=" * 50)

response = agente_com_memoria.run(
    "Qual é o meu nome mesmo? E o que eu faço?",
    session_id="conversa_joao",  # Mesma sessão!
    user_id="joao123"
)

print(response.content)
print("\n🧠 Ele lembrou! A memória está funcionando!")

## 🔍 Explorando as Sessões

Agora vamos dar uma olhada "por baixo do capô" e ver como as sessões ficam armazenadas!

In [ ]:
# Vamos buscar nossa sessão e ver o que tem dentro
sessao = agente_com_memoria.get_session(session_id="conversa_joao")

print("📊 INFORMAÇÕES DA SESSÃO:")
print("=" * 50)
print(f"🆔 ID da Sessão: {sessao.session_id}")
print(f"👤 ID do Usuário: {sessao.user_id}")
print(f"🤖 Nome do Agente: {sessao.agent_id}")
print(f"📝 Número de runs: {len(sessao.runs)}")
print(f"⏰ Criado em: {sessao.created_at}")
print(f"🔄 Atualizado em: {sessao.updated_at}")

In [ ]:
# Vamos ver o histórico completo da conversa
historico = agente_com_memoria.get_chat_history(session_id="conversa_joao")

print("💬 HISTÓRICO COMPLETO DA CONVERSA:")
print("=" * 50)

for i, mensagem in enumerate(historico, 1):
    role_emoji = "👤" if mensagem.role == "user" else "🤖"
    print(f"\n{i}. {role_emoji} {mensagem.role.upper()}:")
    print(f"   {mensagem.content[:100]}...")  # Primeiros 100 caracteres

## 👥 Múltiplos Usuários

Agora vamos ver como o agente consegue distinguir entre diferentes usuários. É como ter conversas separadas no WhatsApp!

In [ ]:
# Vamos simular outro usuário
print("👤 NOVA USUÁRIA: MARIA")
print("=" * 50)

response = agente_com_memoria.run(
    "Oi! Sou a Maria, sou designer e adoro café. Moro no Rio de Janeiro.",
    session_id="conversa_maria",  # Sessão diferente!
    user_id="maria456"  # Usuário diferente!
)

print(response.content)

In [ ]:
# Vamos voltar para o João e ver se não houve confusão
print("🔄 VOLTANDO PARA O JOÃO:")
print("=" * 50)

response = agente_com_memoria.run(
    "Me fala sobre uma pizzaria boa aqui na minha cidade?",
    session_id="conversa_joao",  # Sessão do João
    user_id="joao123"
)

print(response.content)
print("\n🎯 Perfeito! Ele lembrou que o João mora em São Paulo!")

In [ ]:
# E agora vamos perguntar algo para a Maria
print("☕ PERGUNTA PARA A MARIA:")
print("=" * 50)

response = agente_com_memoria.run(
    "Que tipo de café você recomenda?",
    session_id="conversa_maria",  # Sessão da Maria
    user_id="maria456"
)

print(response.content)
print("\n🎨 Ele manteve o contexto da Maria separado do João!")

## 🧠 Memórias de Usuário Avançadas

O Agno tem um sistema de memória ainda mais avançado! Ele pode automaticamente extrair e lembrar informações importantes sobre cada usuário.

É como se o agente tivesse um "caderninho" para cada pessoa, anotando as preferências e características importantes!

In [ ]:
# Criando um agente com memória automática de usuário
agente_memoria_avancada = Agent(
    name="Assistente Memória Pro",
    model=Gemini(id="gemini-2.0-flash-exp"),
    db=db,
    
    # 🧠 Configurações de memória básica
    add_history_to_context=True,
    num_history_runs=3,
    
    # 🎯 MEMÓRIA AUTOMÁTICA DE USUÁRIO!
    enable_user_memories=True,  # Ativa o sistema de memória automática
    add_memories_to_context=True,  # Adiciona as memórias ao contexto
    
    description="Sou um assistente que lembra detalhes importantes sobre cada usuário",
    markdown=True
)

print("🚀 Agente com memória avançada criado!")
print("🧠 Ele vai automaticamente lembrar de detalhes importantes")
print("📝 É como ter um assistente pessoal de verdade!")

In [ ]:
# Vamos conversar e ver como ele extrai memórias
print("💬 CONSTRUINDO MEMÓRIAS AUTOMÁTICAS:")
print("=" * 50)

# Primeira interação - passando várias informações
response = agente_memoria_avancada.run(
    """Oi! Sou o Pedro, tenho 28 anos e trabalho como engenheiro de software. 
    Moro em Belo Horizonte, sou vegetariano e pratico corrida nas manhãs. 
    Tenho dois gatos: Thor e Loki. Adoro ler ficção científica e jogar xadrez.""",
    user_id="pedro789",
    session_id="conversa_pedro"
)

print(response.content)

In [ ]:
# Vamos ver que memórias foram extraídas automaticamente
memorias_pedro = agente_memoria_avancada.get_user_memories(user_id="pedro789")

print("📝 MEMÓRIAS EXTRAÍDAS AUTOMATICAMENTE:")
print("=" * 50)

for i, memoria in enumerate(memorias_pedro, 1):
    print(f"{i}. 🧠 {memoria.content}")
    
print(f"\n✨ Total de memórias: {len(memorias_pedro)}")

In [ ]:
# Agora vamos testar se ele usa essas memórias em novas conversas
print("🔄 TESTANDO O USO DAS MEMÓRIAS:")
print("=" * 50)

# Nova sessão, mas mesmo usuário
response = agente_memoria_avancada.run(
    "Me sugere um restaurante bom aqui na minha cidade?",
    user_id="pedro789",  # Mesmo usuário
    session_id="nova_conversa_pedro"  # Nova sessão!
)

print(response.content)
print("\n🎯 Incrível! Ele lembrou que Pedro é vegetariano e mora em BH!")

## 📊 Visualizando as Sessões

Vamos criar algumas visualizações para entender melhor como as sessões e memórias funcionam!

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

# Vamos listar todas as sessões que temos
usuarios = ["joao123", "maria456", "pedro789"]
dados_sessoes = []

for user_id in usuarios:
    # Aqui normalmente você faria uma query no banco
    # Por simplicidade, vamos simular alguns dados
    if user_id == "joao123":
        dados_sessoes.append({"usuario": "João", "sessoes": 1, "mensagens": 4})
    elif user_id == "maria456":
        dados_sessoes.append({"usuario": "Maria", "sessoes": 1, "mensagens": 2})
    else:
        dados_sessoes.append({"usuario": "Pedro", "sessoes": 2, "mensagens": 3})

df = pd.DataFrame(dados_sessoes)

# Criando gráfico de barras
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Número de sessões por usuário
ax1.bar(df['usuario'], df['sessoes'], color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax1.set_title('📊 Sessões por Usuário', fontsize=14, fontweight='bold')
ax1.set_ylabel('Número de Sessões')
ax1.set_xlabel('Usuários')

# Gráfico 2: Número de mensagens por usuário
ax2.bar(df['usuario'], df['mensagens'], color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax2.set_title('💬 Mensagens por Usuário', fontsize=14, fontweight='bold')
ax2.set_ylabel('Número de Mensagens')
ax2.set_xlabel('Usuários')

plt.tight_layout()
plt.show()

print("📈 Visualização das sessões criada!")
print("🎯 Cada usuário tem suas conversas separadas")

## 🏗️ Diferentes Tipos de Storage

Vamos explorar outros tipos de storage além do SQLite. Cada um tem suas vantagens!

In [ ]:
# Demonstração de diferentes tipos de storage
from agno.db.json import JsonDb
from agno.db.in_memory import InMemoryDb

print("🗃️ TIPOS DE STORAGE DISPONÍVEIS:")
print("=" * 50)

# 1. SQLite (já usamos)
print("1. 📁 SQLite:")
print("   ✅ Perfeito para desenvolvimento")
print("   ✅ Não precisa de servidor")
print("   ✅ Arquivo único")
print("   ❌ Não é ideal para múltiplos usuários simultâneos")

# 2. JSON
print("\n2. 📄 JSON:")
print("   ✅ Super simples")
print("   ✅ Fácil de ler e debugar")
print("   ❌ Não é eficiente para muitos dados")
print("   ❌ Só para protótipos")

# 3. In-Memory
print("\n3. 🧠 In-Memory:")
print("   ✅ Super rápido")
print("   ✅ Perfeito para testes")
print("   ❌ Perde dados quando para o programa")
print("   ❌ Só para desenvolvimento")

# 4. PostgreSQL (produção)
print("\n4. 🐘 PostgreSQL:")
print("   ✅ Perfeito para produção")
print("   ✅ Suporta muitos usuários")
print("   ✅ Muito robusto")
print("   ❌ Precisa de servidor")

In [ ]:
# Vamos testar o JSON storage
json_db = JsonDb(db_file="agente_json.json")

agente_json = Agent(
    name="Agente JSON",
    model=Gemini(id="gemini-2.0-flash-exp"),
    db=json_db,
    add_history_to_context=True,
    description="Agente que usa JSON para storage"
)

# Teste rápido
response = agente_json.run(
    "Oi! Meu nome é Ana e eu gosto de JavaScript!",
    user_id="ana111",
    session_id="teste_json"
)

print("💾 Agente com JSON storage testado!")
print("📄 Os dados ficam salvos em 'agente_json.json'")
print(response.content)

In [ ]:
# Vamos dar uma olhada no arquivo JSON criado
import json

try:
    with open("agente_json.json", "r", encoding="utf-8") as f:
        dados_json = json.load(f)
    
    print("📄 CONTEÚDO DO ARQUIVO JSON:")
    print("=" * 50)
    print(json.dumps(dados_json, indent=2, ensure_ascii=False)[:500] + "...")
    
except FileNotFoundError:
    print("❌ Arquivo JSON ainda não foi criado")
except Exception as e:
    print(f"❌ Erro ao ler arquivo: {e}")

## 🎯 Exercício Prático: Sistema de Atendimento

Vamos criar um sistema completo de atendimento ao cliente com memória! Vai ser como um SAC inteligente que lembra de cada cliente.

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-05_img_02.png)

In [ ]:
# Sistema de Atendimento com Memória
class SistemaAtendimento:
    def __init__(self):
        # Storage para o sistema
        self.db = SqliteDb(db_file="atendimento_cliente.db")
        
        # Agente especializado em atendimento
        self.agente_atendimento = Agent(
            name="Assistente de Atendimento",
            model=Gemini(id="gemini-2.0-flash-exp"),
            db=self.db,
            
            # Configurações de memória
            add_history_to_context=True,
            num_history_runs=10,  # Lembra das últimas 10 interações
            enable_user_memories=True,
            add_memories_to_context=True,
            
            description="""Sou um assistente de atendimento ao cliente especializado.
            Lembro de todos os detalhes dos clientes e suas interações anteriores.
            Sou educado, prestativo e sempre busco resolver os problemas.""",
            
            instructions=[
                "Sempre cumprimente o cliente pelo nome se eu souber",
                "Seja empático e prestativo",
                "Lembre-se de problemas anteriores do cliente",
                "Ofereça soluções práticas",
                "Seja profissional mas amigável"
            ],
            markdown=True
        )
    
    def atender_cliente(self, mensagem, cliente_id, nome_cliente=None):
        """Processa uma mensagem do cliente"""
        session_id = f"atendimento_{cliente_id}"
        
        response = self.agente_atendimento.run(
            mensagem,
            user_id=cliente_id,
            session_id=session_id
        )
        
        return response.content
    
    def historico_cliente(self, cliente_id):
        """Busca o histórico de um cliente"""
        session_id = f"atendimento_{cliente_id}"
        try:
            return self.agente_atendimento.get_chat_history(session_id=session_id)
        except:
            return []
    
    def memorias_cliente(self, cliente_id):
        """Busca as memórias de um cliente"""
        try:
            return self.agente_atendimento.get_user_memories(user_id=cliente_id)
        except:
            return []

# Criando nosso sistema
sistema = SistemaAtendimento()
print("🏢 Sistema de Atendimento criado!")
print("📞 Pronto para atender clientes!")

In [ ]:
# Simulando atendimentos
print("📞 SIMULAÇÃO DE ATENDIMENTOS:")
print("=" * 50)

# Cliente 1: Primeiro contato
print("\n👤 CLIENTE CARLOS - PRIMEIRO CONTATO:")
resposta = sistema.atender_cliente(
    "Oi, sou o Carlos Silva. Comprei um notebook na loja de vocês semana passada e ele não está ligando. Preciso de ajuda!",
    cliente_id="carlos001"
)
print(resposta)

In [ ]:
# Carlos volta no dia seguinte
print("\n🔄 CARLOS RETORNA NO DIA SEGUINTE:")
resposta = sistema.atender_cliente(
    "Oi, sou o Carlos novamente. Testei as sugestões que vocês deram ontem, mas o notebook ainda não liga.",
    cliente_id="carlos001"
)
print(resposta)
print("\n🎯 Ele lembrou do problema anterior do Carlos!")

In [ ]:
# Outro cliente com problema diferente
print("\n👤 CLIENTE FERNANDA - PRIMEIRO CONTATO:")
resposta = sistema.atender_cliente(
    "Olá! Sou a Fernanda. Quero trocar um produto que comprei, mas perdi a nota fiscal. É possível?",
    cliente_id="fernanda002"
)
print(resposta)

In [ ]:
# Vamos ver as memórias que foram criadas
print("🧠 MEMÓRIAS DO CARLOS:")
print("=" * 50)
memorias_carlos = sistema.memorias_cliente("carlos001")
for memoria in memorias_carlos:
    print(f"📝 {memoria.content}")

print("\n🧠 MEMÓRIAS DA FERNANDA:")
print("=" * 50)
memorias_fernanda = sistema.memorias_cliente("fernanda002")
for memoria in memorias_fernanda:
    print(f"📝 {memoria.content}")

## 🏆 Desafio Final: Chat Bot Personalizado

Agora é sua vez! Vamos criar um chatbot que:
1. 🧠 Lembra das preferências de cada usuário
2. 🎯 Personaliza as respostas baseado no histórico
3. 📊 Mantém estatísticas de uso
4. 💾 Usa o storage de sua escolha

In [ ]:
# Seu desafio: Complete este chatbot personalizado!
class ChatBotPersonalizado:
    def __init__(self, nome_bot="MeuBot"):
        # TODO: Configure o storage
        self.db = SqliteDb(db_file="meu_chatbot.db")
        
        # TODO: Configure o agente com memória
        self.agente = Agent(
            name=nome_bot,
            model=Gemini(id="gemini-2.0-flash-exp"),
            db=self.db,
            # Adicione suas configurações aqui!
            add_history_to_context=True,
            enable_user_memories=True,
            add_memories_to_context=True,
            description="Sou um chatbot personalizado que aprende sobre cada usuário!",
            markdown=True
        )
        
        self.estatisticas = {
            "total_conversas": 0,
            "usuarios_unicos": set(),
            "mensagens_total": 0
        }
    
    def conversar(self, mensagem, usuario_id):
        """Processa uma mensagem do usuário"""
        # Atualiza estatísticas
        self.estatisticas["mensagens_total"] += 1
        self.estatisticas["usuarios_unicos"].add(usuario_id)
        
        # TODO: Implemente a lógica de conversação
        session_id = f"chat_{usuario_id}"
        
        response = self.agente.run(
            mensagem,
            user_id=usuario_id,
            session_id=session_id
        )
        
        return response.content
    
    def mostrar_estatisticas(self):
        """Mostra estatísticas do bot"""
        print("📊 ESTATÍSTICAS DO CHATBOT:")
        print("=" * 40)
        print(f"💬 Total de mensagens: {self.estatisticas['mensagens_total']}")
        print(f"👥 Usuários únicos: {len(self.estatisticas['usuarios_unicos'])}")
        print(f"📈 Média de mensagens por usuário: {self.estatisticas['mensagens_total'] / max(len(self.estatisticas['usuarios_unicos']), 1):.1f}")

# Teste seu chatbot!
meu_bot = ChatBotPersonalizado("AssistenteIA")
print("🤖 Seu chatbot personalizado foi criado!")
print("✨ Agora teste com diferentes usuários!")

In [ ]:
# Teste seu chatbot com diferentes usuários
print("🧪 TESTANDO O CHATBOT PERSONALIZADO:")
print("=" * 50)

# Usuário 1
resposta = meu_bot.conversar(
    "Oi! Sou o Lucas, tenho 25 anos e adoro tecnologia. Trabalho com IA.",
    "lucas123"
)
print(f"🤖 Bot: {resposta}")

# Usuário 2  
resposta = meu_bot.conversar(
    "Olá! Sou a Sofia, professora de matemática. Gosto de explicar coisas complexas de forma simples.",
    "sofia456"
)
print(f"\n🤖 Bot: {resposta}")

# Lucas volta
resposta = meu_bot.conversar(
    "Me explica como funciona a memória em sistemas de IA?",
    "lucas123"
)
print(f"\n🤖 Bot para Lucas: {resposta}")

# Estatísticas
print("\n")
meu_bot.mostrar_estatisticas()

## 📈 Métricas e Monitoramento

Vamos ver como monitorar o desempenho do nosso sistema de memória!

In [ ]:
# Analisando métricas das sessões
import matplotlib.pyplot as plt
import numpy as np

# Simulando dados de métricas
usuarios = ['João', 'Maria', 'Pedro', 'Carlos', 'Fernanda', 'Lucas', 'Sofia']
mensagens_por_usuario = [4, 2, 3, 4, 2, 2, 1]
sessoes_por_usuario = [1, 1, 2, 2, 1, 1, 1]
memorias_por_usuario = [3, 2, 5, 4, 2, 3, 2]

# Criando dashboard de métricas
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Mensagens por usuário
bars1 = ax1.bar(usuarios, mensagens_por_usuario, color='skyblue', alpha=0.8)
ax1.set_title('💬 Mensagens por Usuário', fontsize=14, fontweight='bold')
ax1.set_ylabel('Número de Mensagens')
ax1.tick_params(axis='x', rotation=45)
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{int(height)}', ha='center', va='bottom')

# Gráfico 2: Sessões por usuário
bars2 = ax2.bar(usuarios, sessoes_por_usuario, color='lightgreen', alpha=0.8)
ax2.set_title('📊 Sessões por Usuário', fontsize=14, fontweight='bold')
ax2.set_ylabel('Número de Sessões')
ax2.tick_params(axis='x', rotation=45)
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{int(height)}', ha='center', va='bottom')

# Gráfico 3: Memórias por usuário
bars3 = ax3.bar(usuarios, memorias_por_usuario, color='salmon', alpha=0.8)
ax3.set_title('🧠 Memórias por Usuário', fontsize=14, fontweight='bold')
ax3.set_ylabel('Número de Memórias')
ax3.tick_params(axis='x', rotation=45)
for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{int(height)}', ha='center', va='bottom')

# Gráfico 4: Distribuição de atividade
atividade_total = np.array(mensagens_por_usuario) + np.array(memorias_por_usuario)
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#98D8C8']
ax4.pie(atividade_total, labels=usuarios, autopct='%1.1f%%', colors=colors, startangle=90)
ax4.set_title('📈 Distribuição de Atividade Total', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Dashboard de métricas criado!")
print("🎯 Agora você pode monitorar como seus agentes estão sendo usados")

## 🎓 Resumo do Módulo 5

Parabéns! 🎉 Você completou o Módulo 5 e agora sabe como criar agentes com memória de longo prazo!

### 🧠 O que aprendemos:

1. **💾 Storage e Persistência**
   - Como configurar diferentes tipos de bancos de dados
   - SQLite, JSON, PostgreSQL e outros
   - Quando usar cada tipo

2. **📝 Sessões e Runs**
   - Como funcionam as conversas persistentes
   - Separação entre diferentes usuários
   - Histórico de mensagens

3. **🧠 Sistema de Memória**
   - Memória automática de usuários
   - Extração de informações importantes
   - Personalização de respostas

4. **👥 Múltiplos Usuários**
   - Como gerenciar diferentes usuários
   - Isolamento de dados
   - Experiências personalizadas

5. **📊 Monitoramento**
   - Métricas de uso
   - Análise de conversas
   - Dashboard de acompanhamento

### 🚀 Próximos Passos:

No **Módulo 6**, vamos aprender sobre **Agentes Multimodal**! Vamos ensinar nossos agentes a:
- 🖼️ Processar imagens
- 🎵 Entender áudio
- 🎥 Analisar vídeos
- 📄 Trabalhar com diferentes tipos de arquivos

**Dica!** 💡 Pratique criando diferentes tipos de agentes com memória. Cada caso de uso pode precisar de uma configuração diferente!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-05_img_03.png)

## 🏋️‍♂️ Exercícios de Fixação

Agora é hora de praticar! Complete estes exercícios para consolidar seu aprendizado:

In [ ]:
# EXERCÍCIO 1: Crie um agente especialista em culinária que lembra das preferências alimentares
# TODO: Complete o código abaixo

chef_bot = Agent(
    name="Chef Virtual",
    model=Gemini(id="gemini-2.0-flash-exp"),
    # Adicione suas configurações aqui
    db=SqliteDb(db_file="chef_bot.db"),
    add_history_to_context=True,
    enable_user_memories=True,
    description="Sou um chef virtual que lembra das preferências culinárias de cada pessoa",
    markdown=True
)

# Teste com um usuário que é vegetariano
resposta = chef_bot.run(
    "Oi! Sou vegetariano e adoro comida italiana. Me sugere uma receita?",
    user_id="chef_user1",
    session_id="culinaria1"
)

print("👨‍🍳 Chef Bot:")
print(resposta.content)

# Agora teste se ele lembra em uma nova conversa
resposta2 = chef_bot.run(
    "Quero fazer algo diferente hoje, mas que combine comigo.",
    user_id="chef_user1",
    session_id="culinaria2"  # Nova sessão!
)

print("\n🔄 Em nova sessão:")
print(resposta2.content)

In [ ]:
# EXERCÍCIO 2: Sistema de recomendação de filmes com memória
# Complete a implementação

class RecomendadorFilmes:
    def __init__(self):
        # TODO: Configure o agente
        self.db = SqliteDb(db_file="filmes_bot.db")
        
        self.agente = Agent(
            name="Cinéfilo IA",
            model=Gemini(id="gemini-2.0-flash-exp"),
            db=self.db,
            add_history_to_context=True,
            enable_user_memories=True,
            add_memories_to_context=True,
            description="""Sou um especialista em cinema que lembra dos gostos de cada pessoa.
            Recomendo filmes baseado no histórico e preferências do usuário.""",
            instructions=[
                "Lembre-se dos gêneros favoritos de cada usuário",
                "Considere filmes já assistidos",
                "Faça recomendações personalizadas",
                "Explique por que está recomendando"
            ],
            markdown=True
        )
    
    def recomendar(self, mensagem, usuario_id):
        session_id = f"filmes_{usuario_id}"
        response = self.agente.run(
            mensagem,
            user_id=usuario_id,
            session_id=session_id
        )
        return response.content

# Teste o sistema
cinema_bot = RecomendadorFilmes()

print("🎬 SISTEMA DE RECOMENDAÇÃO DE FILMES:")
print("=" * 50)

# Usuário conta suas preferências
resposta = cinema_bot.recomendar(
    "Oi! Adoro filmes de ficção científica e ação. Já assisti todos os filmes da Marvel. Odeio romance.",
    "cinefilo001"
)
print(resposta)

# Pedindo nova recomendação
resposta = cinema_bot.recomendar(
    "Me recomenda algo para assistir hoje à noite?",
    "cinefilo001"
)
print(f"\n🎯 Recomendação personalizada:\n{resposta}")

In [ ]:
# EXERCÍCIO 3: Análise de performance do sistema de memória
# Complete a função de análise

def analisar_performance_memoria(agente, usuarios_teste):
    """Analisa como o sistema de memória está performando"""
    
    resultados = {
        "usuarios_analisados": len(usuarios_teste),
        "total_memorias": 0,
        "total_sessoes": 0,
        "memoria_por_usuario": {}
    }
    
    for usuario_id in usuarios_teste:
        try:
            # TODO: Busque as memórias do usuário
            memorias = agente.get_user_memories(user_id=usuario_id)
            num_memorias = len(memorias)
            
            # TODO: Conte as sessões (aqui simplificamos)
            num_sessoes = 1  # Simplificado para o exercício
            
            resultados["total_memorias"] += num_memorias
            resultados["total_sessoes"] += num_sessoes
            resultados["memoria_por_usuario"][usuario_id] = num_memorias
            
        except Exception as e:
            print(f"❌ Erro ao analisar usuário {usuario_id}: {e}")
    
    return resultados

# Teste a análise
usuarios_para_testar = ["joao123", "maria456", "pedro789"]
analise = analisar_performance_memoria(agente_memoria_avancada, usuarios_para_testar)

print("📊 ANÁLISE DE PERFORMANCE DA MEMÓRIA:")
print("=" * 50)
print(f"👥 Usuários analisados: {analise['usuarios_analisados']}")
print(f"🧠 Total de memórias: {analise['total_memorias']}")
print(f"📊 Total de sessões: {analise['total_sessoes']}")
print(f"📈 Média de memórias por usuário: {analise['total_memorias'] / max(analise['usuarios_analisados'], 1):.1f}")

print("\n🔍 Detalhes por usuário:")
for usuario, memorias in analise['memoria_por_usuario'].items():
    print(f"  {usuario}: {memorias} memórias")

## 🎯 Parabéns!

Você completou todos os exercícios do **Módulo 5: Memória e Persistência**! 🎉

Agora você sabe como:
- ✅ Configurar diferentes tipos de storage
- ✅ Criar agentes com memória persistente
- ✅ Gerenciar múltiplos usuários
- ✅ Implementar sistemas de memória automática
- ✅ Monitorar performance do sistema

**Continue praticando** e no próximo módulo vamos aprender sobre **Agentes Multimodal**! 🚀

**Dica Final!** 💡 A memória é fundamental para criar experiências de usuário incríveis. Use ela para tornar seus agentes mais inteligentes e personalizados!

---

**Próximo:** Módulo 6 - Agentes Multimodal 🖼️🎵🎥